In [ ]:
import tensorflow as tf

print(tf.__version__)

In [ ]:
import numpy as np
import nibabel as nib
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Input, Conv3D, MaxPooling3D, Flatten, Dense, Dropout, BatchNormalization, Activation
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import os


In [ ]:
def create_model(input_shape):
    model = Sequential([
        Input(shape=input_shape),

        # Camada 1
        Conv3D(32, (3, 3, 3), padding='same'),
        BatchNormalization(),
        Activation('elu'),
        MaxPooling3D(pool_size=(2, 2, 2), padding='same'),

        # Camada 2
        Conv3D(64, (3, 3, 3), padding='same'),
        BatchNormalization(),
        Activation('elu'),
        MaxPooling3D(pool_size=(2, 2, 2), padding='same'),

        # Camada 3
        Conv3D(128, (3, 3, 3), padding='same'),
        BatchNormalization(),
        Activation('elu'),
        MaxPooling3D(pool_size=(2, 2, 2), padding='same'),

        # Camada densa com redução de neurônios
        Flatten(),
        Dense(256, activation='elu'),  # Reduzido de 512 para 256
        BatchNormalization(),
        Dropout(0.5),

        Dense(128, activation='elu'),  # Reduzido de 512 para 128
        BatchNormalization(),
        Dropout(0.5),

        # Camada de saída
        Dense(2, activation='softmax')  # 2 classes: CN, AD
    ])
    
    return model

# Função para carregar imagens NIfTI, seus rótulos e cortar as imagens
def load_nifti_images_with_labels(base_dir, slice_idx):
    images = []
    labels = []
    
    # Caminhos das subpastas
    for label in ['cn', 'ad']:
        label_dir = os.path.join(base_dir, label)
        for fname in os.listdir(label_dir):
            img_path = os.path.join(label_dir, fname)
            img = nib.load(img_path).get_fdata()
            img_cropped = img[slice_idx[0], slice_idx[1], slice_idx[2]]
            images.append(img_cropped)
            labels.append(label)

    # Codificando os rótulos
    label_encoder = LabelEncoder()
    labels_encoded = label_encoder.fit_transform(labels)
    
    return np.array(images), labels_encoded, label_encoder.classes_

# Função para carregar imagens NIfTI, seus rótulos e cortar as imagens
def load_nifti_paths(base_dir):
    image_paths = []
    labels = []
    
    # Caminhos das subpastas
    for label in ['cn', 'ad']:
        label_dir = os.path.join(base_dir, label)
        for fname in os.listdir(label_dir):
            img_path = os.path.join(label_dir, fname)
            image_paths.append(img_path)
            labels.append(label)

    # Codificando os rótulos
    label_encoder = LabelEncoder()
    labels_encoded = label_encoder.fit_transform(labels)

    # Transformando os rótulos para one-hot encoding
    labels_one_hot = to_categorical(labels_encoded, num_classes=2)
    
    return image_paths, labels_one_hot

# Função para carregar imagens NIfTI, seus rótulos e cortar as imagens
def nifti_data_generator(image_paths, labels, batch_size, slice_idx):
    while True:
        for i in range(0, len(image_paths), batch_size):
            batch_paths = image_paths[i:i + batch_size]
            batch_labels = labels[i:i + batch_size]
            images = []
            
            for path in batch_paths:
                img = nib.load(path).get_fdata()
                img_cropped = img[slice_idx[0], slice_idx[1], slice_idx[2]]
                images.append(img_cropped)
            
            images = np.array(images).reshape((-1, *img_cropped.shape, 1))
            batch_labels = np.array(batch_labels)

            # Embaralhar os dados
            images, batch_labels = shuffle(images, batch_labels, random_state=42)
            yield images, batch_labels



In [ ]:
base_dir = os.path.abspath("C:/Users/Team Taiane/Desktop/ADNI/Lil_adni/ADNI1_ADNI2")

# Definindo caminhos
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'validation')
results_dir = os.path.join(base_dir, 'results')

# Índices de corte para o corte NIfTI
SLICE_NII_IDX0 = slice(24, 169)
SLICE_NII_IDX1 = slice(24, 206)
SLICE_NII_IDX2 = slice(6, 161)

# Passando os índices de corte (como uma tupla) para a função geradora
crop_slices = (SLICE_NII_IDX0, SLICE_NII_IDX1, SLICE_NII_IDX2)

# Criar o diretório de resultados se ele não existir
os.makedirs(results_dir, exist_ok=True)

In [ ]:
# Carregar os dados com cortes
train_images, train_labels, class_labels = load_nifti_images_with_labels(train_dir, 
                                                                        (SLICE_NII_IDX0, SLICE_NII_IDX1, SLICE_NII_IDX2))
val_images, val_labels, _ = load_nifti_images_with_labels(val_dir, 
                                                           (SLICE_NII_IDX0, SLICE_NII_IDX1, SLICE_NII_IDX2))

# Adicionar a dimensão do canal
train_images = train_images.reshape((-1, *train_images.shape[1:], 1))
val_images = val_images.reshape((-1, *val_images.shape[1:], 1))

# Embaralhar os dados
train_images, train_labels = shuffle(train_images, train_labels, random_state=42)
val_images, val_labels = shuffle(val_images, val_labels, random_state=42)

In [ ]:
train_image_paths, train_labels = load_nifti_paths(train_dir)
val_image_paths, val_labels = load_nifti_paths(val_dir)

batch_size = 8
slice_idx = [SLICE_NII_IDX0, SLICE_NII_IDX1, SLICE_NII_IDX2]
steps_per_epoch = len(train_image_paths) // batch_size
validation_steps = len(val_image_paths) // batch_size

train_generator = nifti_data_generator(train_image_paths, train_labels, batch_size, slice_idx)
val_generator = nifti_data_generator(val_image_paths, val_labels, batch_size, slice_idx)

In [ ]:
# Compila model
input_shape = (145, 182, 155, 1)
model = create_model(input_shape)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

In [ ]:
n = len(os.listdir(results_dir))
folder_name = "test_" + str(n+1)
results_dir = os.path.join(results_dir, folder_name)

os.makedirs(results_dir, exist_ok=True)

In [ ]:
# Treinamento
history = model.fit(
    train_generator,
    epochs=50,
    verbose=1,
    validation_data=val_generator,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    batch_size=16
)

# Salva o modelo, arquitetura, pesos e estado do otimizador
model.save(os.path.join(results_dir, "binary_classifier.h5"))

In [ ]:
# Função para avaliação
def evaluate_model(val_images, val_labels, model, results_dir):
    predictions = model.predict(val_images)
    predicted_classes = np.argmax(predictions, axis=1)

    report = classification_report(val_labels, predicted_classes, target_names=class_labels, output_dict=True)
    
    with open(os.path.join(results_dir, "classification_report.txt"), "w") as f:
        f.write(classification_report(val_labels, predicted_classes, target_names=class_labels))
    print("Classification Report:\n", classification_report(val_labels, predicted_classes, target_names=class_labels))
    
    conf_matrix = confusion_matrix(val_labels, predicted_classes)
    
    # Normalizar a matriz de confusão para porcentagens
    conf_matrix_normalized = conf_matrix.astype('float') / conf_matrix.sum(axis=1)[:, np.newaxis]
    np.savetxt(os.path.join(results_dir, "confusion_matrix.txt"), conf_matrix, fmt='%d')
    
    plt.figure(figsize=(8, 6))
    plt.imshow(conf_matrix_normalized, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Confusion Matrix')
    plt.colorbar()
    
    tick_marks = np.arange(len(class_labels))
    plt.xticks(tick_marks, class_labels, rotation=45)
    plt.yticks(tick_marks, class_labels)
    
    # Adicionando os rótulos com porcentagens
    thresh = conf_matrix_normalized.max() / 2.
    for i, j in np.ndindex(conf_matrix_normalized.shape):
        plt.text(j, i, f'{conf_matrix_normalized[i, j]:.2%}', 
                 horizontalalignment="center",
                 color="white" if conf_matrix_normalized[i, j] > thresh else "black")
    
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    plt.savefig(os.path.join(results_dir, "confusion_matrix.png"))
    plt.close()

    # Exibindo as métricas
    accuracy = report['accuracy']
    recall = report['weighted avg']['recall']
    f1_score = report['weighted avg']['f1-score']

    print(f"Acurácia: {accuracy:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1_score:.4f}")

evaluate_model(val_images, val_labels, model, results_dir)

In [ ]:
# Função para plotar e salvar o histórico de treinamento
def plot_history(history, results_dir):
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Acurácia de Treinamento')
    plt.plot(history.history['val_accuracy'], label='Acurácia de Validação')
    plt.title('Acurácia ao Longo das Épocas')
    plt.xlabel('Épocas')
    plt.ylabel('Acurácia')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Perda de Treinamento')
    plt.plot(history.history['val_loss'], label='Perda de Validação')
    plt.title('Perda ao Longo das Épocas')
    plt.xlabel('Épocas')
    plt.ylabel('Perda')
    plt.legend()

    plt.tight_layout()
    plt.savefig(os.path.join(results_dir, "training_history.png"))
    plt.close()

plot_history(history, results_dir)